In [1]:
#export
import sys
sys.path.append('../')

In [2]:
#export
from datasets import get_dataset
from path import Path
from models  import get_model
from sklearn.metrics import accuracy_score
import torch
import os
#from torch.utils.tensorboard import SummaryWriter

In [3]:
import argparse

In [4]:
data_path = Path('../../filtered_ds/')

In [5]:
parser = argparse.ArgumentParser(description='')

In [6]:
parser.add_argument('--dataset', default='video', type=str,
                        help='name of dataset to train upon')
parser.add_argument('--root_dir', default=Path('../filtered_ds/'), type=Path,
                        help='name of dataset to train upon')
parser.add_argument('--csv_file', default='videos_df', type=str,
                        help='name of dataset to train upon')
parser.add_argument('--crop_size', default=224, type=int,
                        help='name of dataset to train upon')
parser.add_argument('--batch_size', default=4, type=int,
                        help='name of dataset to train upon')
parser.add_argument('--workers', default=4, type=int,
                        help='name of dataset to train upon')
parser.add_argument('--embed_dim', default=512, type=int,
                        help='name of dataset to train upon')
parser.add_argument('--pretrained', default=True, type=bool,
                        help='name of dataset to train upon')
parser.add_argument('--dropout', default=0.3, type=float,
                        help='name of dataset to train upon')
parser.add_argument('--model_name', default='resnet50', type=str,
                        help='name of dataset to train upon')
parser.add_argument('--num_classes', default=2, type=int,
                        help='name of dataset to train upon')
parser.add_argument('--lr', default=1e-3, type=float,
                        help='name of dataset to train upon')
parser.add_argument('--epochs', default=50, type=int,
                        help='name of dataset to train upon')
parser.add_argument('--log_interval', default=10, type=int,
                        help='name of dataset to train upon')
parser.add_argument('--save_model_path', default='./checkpoints/', type=str,
                        help='name of dataset to train upon')






_StoreAction(option_strings=['--save_model_path'], dest='save_model_path', nargs=None, const=None, default='./checkpoints/', type=<class 'str'>, choices=None, help='name of dataset to train upon', metavar=None)

In [7]:
args = parser.parse_args("--dataset video --root_dir ../../filtered_ds/ --csv_file videos_df\
                          --crop_size 224 --batch_size 4 --workers 4 --embed_dim 512\
                          --pretrained True --dropout 0.2 \
                          --model_name resnet34 --num_classes 2  --lr 1e-3\
                          --epochs 50 --log_interval 10\
                          ".split())

In [8]:
ds = get_dataset(args)

In [9]:
args.dataset

'video'

In [10]:
#export
train_loader = torch.utils.data.DataLoader(ds(csv_file=args.root_dir/args.csv_file, root_dir=args.root_dir, split='train', 
        phase='train', crop_size=args.crop_size), 
        batch_size=args.batch_size, shuffle=False, num_workers=args.workers, pin_memory=True)
val_loader = torch.utils.data.DataLoader(ds(csv_file=args.root_dir/args.csv_file, root_dir=args.root_dir, split='test',
        phase='test', crop_size=args.crop_size), batch_size=args.batch_size,
        shuffle=False, num_workers=args.workers, pin_memory=True)


In [11]:
#export
use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu")  

In [12]:
next(iter(train_loader))[0].size()

torch.Size([4, 30, 3, 224, 224])

In [13]:
#export
cnn_encoder, rnn_decoder = get_model(args)

Fetching model resnet34


In [14]:
res=rnn_decoder(cnn_encoder(next(iter(train_loader))[0]))
res.size()

torch.Size([4, 2])

In [15]:
#export
print("Using", torch.cuda.device_count(), "GPU!")
# Combine all EncoderCNN + DecoderRNN parameters
crnn_params = list(cnn_encoder.fc1.parameters()) + list(cnn_encoder.bn1.parameters()) + \
              list(cnn_encoder.fc2.parameters()) + list(cnn_encoder.bn2.parameters()) + \
              list(cnn_encoder.fc3.parameters()) + list(rnn_decoder.parameters())

Using 0 GPU!


In [16]:
#export
optimizer = torch.optim.Adam(crnn_params, lr=args.lr)

In [18]:
#export
#cnn_encoder, rnn_decoder = model
epoch_train_losses = []
epoch_train_scores = []
epoch_test_losses = []
epoch_test_scores = []

for epoch in range(args.epochs):
    cnn_encoder.train()
    rnn_decoder.train()

    losses = []
    scores = []
    N_count = 0   # counting total trained sample in one epoch
    for batch_idx, (X, y) in enumerate(train_loader):
        # distribute data to device
        X, y = X.to(device), y.to(device).view(-1, )

        N_count += X.size(0)

        optimizer.zero_grad()
        output = rnn_decoder(cnn_encoder(X))   # output has dim = (batch, number of classes)

        loss = F.cross_entropy(output, y)
        losses.append(loss.item())

        # to compute accuracy
        y_pred = torch.max(output, 1)[1]  # y_pred != output
        step_score = accuracy_score(y.cpu().data.squeeze().numpy(), y_pred.cpu().data.squeeze().numpy())
        scores.append(step_score)         # computed on CPU

        loss.backward()
        optimizer.step()

        # show information
        if (batch_idx + 1) % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accu: {:.2f}%'.format(
                epoch + 1, N_count, len(train_loader.dataset), 100. * (batch_idx + 1) / len(train_loader), loss.item(), 100 * step_score))
            
    save_model_path=''
    cnn_encoder.eval()
    rnn_decoder.eval()

    test_loss = 0
    all_y = []
    all_y_pred = []
    with torch.no_grad():
        for X, y in val_loader:
            # distribute data to device
            X, y = X.to(device), y.to(device).view(-1, )

            output = rnn_decoder(cnn_encoder(X))

            loss = F.cross_entropy(output, y, reduction='sum')
            test_loss += loss.item()                 # sum up batch loss
            y_pred = output.max(1, keepdim=True)[1]  # (y_pred != output) get the index of the max log-probability

            # collect all y and y_pred in all batches
            all_y.extend(y)
            all_y_pred.extend(y_pred)

    test_loss /= len(val_loader.dataset)

    # compute accuracy
    all_y = torch.stack(all_y, dim=0)
    all_y_pred = torch.stack(all_y_pred, dim=0)
    test_score = accuracy_score(all_y.cpu().data.squeeze().numpy(), all_y_pred.cpu().data.squeeze().numpy())
    
    # show information
    print('\nTest set ({:d} samples): Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(len(all_y), test_loss, 100* test_score))
    train_losses, train_scores = losses, scores
    epoch_test_loss, epoch_test_score = test_loss, test_score
    
    epoch_train_losses.append(train_losses)
    epoch_train_scores.append(train_scores)
    epoch_test_losses.append(epoch_test_loss)
    epoch_test_scores.append(epoch_test_score)

    # save all train test results
    A = np.array(epoch_train_losses)
    B = np.array(epoch_train_scores)
    C = np.array(epoch_test_losses)
    D = np.array(epoch_test_scores)
    np.save(os.path.join(args.save_model_path, './epoch_training_losses.npy'), A)
    np.save(os.path.join(args.save_model_path, './epoch_training_scores.npy', B))
    np.save(os.path.join(args.save_model_path, './epoch_test_loss.npy', C))
    np.save(os.path.join(args.save_model_path, './epoch_test_score.npy', D))
    
    # save Pytorch models of best record
    torch.save(cnn_encoder.state_dict(), os.path.join(args.save_model_path, 'cnn_encoder_epoch{}.pth'.format(epoch + 1)))  # save spatial_encoder
    torch.save(rnn_decoder.state_dict(), os.path.join(args.save_model_path, 'rnn_decoder_epoch{}.pth'.format(epoch + 1)))  # save motion_encoder
    torch.save(optimizer.state_dict(), os.path.join(args.save_model_path, 'optimizer_epoch{}.pth'.format(epoch + 1)))      # save optimizer
    print("Epoch {} model saved!".format(epoch + 1))
    

KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  Fi

KeyboardInterrupt: 

In [20]:
next(iter(val_loader))[0][1].size()

torch.Size([30, 3, 160, 160])

In [19]:
!python notebook2script.py 02_train.ipynb

Converted 02_train.ipynb to exp/nb_02.py
